In [ ]:
# Importing Packages
import pandas as pd
import numpy as np
import statistics
pd.set_option("display.max_columns", None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/Marwan Albanna')

Mounted at /content/drive


### Loading Data

In [ ]:
# df_8_Life = pd.read_excel("/content/8_reduced.xlsx",sheet_name = "Life")
# df_8_PreHeat = pd.read_excel("/content/8_reduced.xlsx",sheet_name = "PreHeat")
# df_8_Informations = pd.read_excel("/content/8_reduced.xlsx",sheet_name = "Informations")
# df_8_ActionList = pd.read_excel("/content/8_reduced.xlsx",sheet_name = "Action List")

In [ ]:
# df_cell_comparisons = pd.read_excel("/content/drive/MyDrive/nitesh /Cell Comparisons JA vs AT vs Techno - Copy.xlsx",sheet_name = "Raw")

In [ ]:
# df_halo_updated_by_cut_1 = pd.read_excel("/content/drive/MyDrive/nitesh /HALO 2.xlsx",sheet_name = "By Cut 1")
# df_halo_updated_PGS = pd.read_excel("/content/drive/MyDrive/nitesh /HALO 2.xlsx",sheet_name = "PGS")

### Data Processing

In [ ]:
# Here we can see the Cell name - 1A008
# cellName = df_8_Informations.value[1]
# df_8_Informations

In [ ]:
import os
import openpyxl

In [ ]:
df_cell_comparisons = pd.read_excel("/content/drive/MyDrive/nitesh /Cell Comparisons JA vs AT vs Techno - Copy.xlsx",sheet_name = "Raw")
df_halo_updated_by_cut_1 = pd.read_excel("/content/drive/MyDrive/nitesh /HALO 2.xlsx",sheet_name = "By Cut 1")
df_halo_updated_PGS = pd.read_excel("/content/drive/MyDrive/nitesh /HALO 2.xlsx",sheet_name = "PGS")

In [ ]:
# Converting CT CELL ID value from 8 to 1A008
def convert_ct_cell_value(df,value):
    df["CT_CELL_ID"] = value
# convert_ct_cell_value(df_8_Life,cellName) #hard coded value

In [ ]:
from datetime import datetime
def datetime2year(dt): 
    format_data = "%Y-%m-%d %H:%M:%S.%f"
    format_data2 = "%Y-%m-%d %H:%M:%S"
    try:
      dt = datetime.strptime(str(dt), format_data)
    except:
      try:
        dt = datetime.strptime(str(dt), format_data2)
      except:
        return dt
    temp = datetime(1899, 12, 30)
    delta = dt - temp
    t = float(delta.days) + (float(delta.seconds) / 86400)
    return t

In [ ]:
import math
def getPotAge(l):
  l = l.iloc[:,7]
  s = [i for i in l if i==i and str(i).isnumeric()]
  print(s)
  s = np.unique(s)
  s = list(sorted(s))
  ans = list([np.nan])
  for i in s[:10]:
    ans.append(i);
  s.reverse()
  for i in range(min(len(s)-1, 10)):
    if abs(s[i+1] - s[i])>=1000:
      ans.append(s[i]);

  print(ans)
  return ans;

In [ ]:
def conversion_df_Life_to_halo(df_life,df_halo):
#no age filter
    pot_age = getPotAge(df_life);
    df_new = df_life.copy()
    df_new = df_new[df_new.CD_POT_AGE.apply(lambda x : True if x in pot_age else False )]
    temp_columns = df_halo.columns[:2]
    print(temp_columns)
    try :
      df_new.insert(loc = 0,
            column = temp_columns[1],
            value = "")
    except:
      pass
    try:
      df_new.insert(loc = 0,
            column = temp_columns[0],
            value = "")
    except:
      pass
     #df_8 updation not done 10 top and null ... bottom 10 with unique
    # print(df_new.head())
    for i in range(1,len(df_halo.columns[53:])):
        df_new.insert(loc = 52+i,
          column = df_halo.columns[53+i],
          value = "")
    df_halo = df_halo.append(df_life, ignore_index=True)
    # print(df_halo.tail())
    return df_halo
# df_halo_updated = conversion_df_Life_to_halo(df_8_Life,df_halo_updated_by_cut_1)

In [ ]:
def cleaning(df_halo_updated, flag, cellName):
    df_halo_updated.columns = [x.replace('\n','') for x in df_halo_updated.columns]
    df_temp = df_halo_updated[df_halo_updated["CT_CELL_ID"] == cellName]
    df_others = df_halo_updated[df_halo_updated["CT_CELL_ID"] != cellName]
    df_temp["Factory"] = "AT" if 'A' in cellName else "DX"   #add filter
    
    for i in df_temp["CT_CELL_ID"].index:
        df_temp["TAGS 2"] = str(df_temp["Time From Cut-in"][i])+"_"+df_temp["CT_CELL_ID"][i]
        df_temp["TAGS"] = str(datetime2year(df_temp["Date"][i]))+"_"+df_temp["CT_CELL_ID"][i]
    TE = df_cell_comparisons[df_cell_comparisons["Cell"] == cellName]["TE"].values[0]
    DE = df_cell_comparisons[df_cell_comparisons["Cell"] == cellName]["DE"].values[0]
    df_temp["Average Temperature (TE/DE)"] = np.ceil((TE+DE)/2)
    df_halo_updated.iloc[:,np.r_[0,55:68]] = pd.merge(df_halo_updated.iloc[:,np.r_[0,55:68]], df_halo_updated_PGS.iloc[:,np.r_[0,5:19]],how ='left', left_on = "TAGS 2", right_on = "TAGS")#1
    index = list(df_8_ActionList.columns).index("CD_POT_AGE")
    df_temp.iloc[:,68:84] = df_8_ActionList.iloc[:,index:index + 18] #2
    df_temp.iloc[:,84] = df_temp.iloc[:,69]+df_temp.iloc[:,71]
    df_temp.iloc[:,85] = df_temp.iloc[:,70]+df_temp.iloc[:,72] 
    df_temp.iloc[:,87] = "L1" #did not find 
    df_temp[cellName + " Bath up Forecast"] = list(df_halo_updated_PGS[df_halo_updated_PGS["Cell"] == cellName].iloc[:,19])[-1]
    df_temp[cellName + " Temperature"] = statistics.mean(list(df_halo_updated_PGS[df_halo_updated_PGS["Cell"] == cellName].iloc[:,16])[:-1])
    df_temp[cellName + " PH Temerature"] = df_temp["Forecasted Temperature"]
    df_temp[cellName + " TGT"] = ""
    df_temp[cellName + " TMT"] = ""
    df_temp["Voltage"] = df_temp["CT_RAW_VOLT"].iloc[5]
    df_out = df_others.append(df_temp, ignore_index=True)
    df_out.drop(['Paste Type', 'Collector Bar','Anode CV %', 'Technology'],axis = 1,inplace=True)
    if flag :
      df_out.to_csv(f"halo2_final_bycut1.csv",index=False)    
    return df_out

# cleaning(df_halo_updated)

In [ ]:
l = ["/content/8_reduced.xlsx"]
for i, f in enumerate(l):
  df_8_Life = pd.read_excel(f,sheet_name = "Life")
  df_8_PreHeat = pd.read_excel(f,sheet_name = "PreHeat")
  df_8_Informations = pd.read_excel(f,sheet_name = "Informations")
  df_8_ActionList = pd.read_excel(f,sheet_name = "Action List")
  cellName = df_8_Informations.value[1]
  convert_ct_cell_value(df_8_Life,cellName)
  df_halo_updated = conversion_df_Life_to_halo(df_8_Life,df_halo_updated_by_cut_1)
  df_halo_updated_by_cut_1 = cleaning(df_halo_updated, i==(len(l)-1), cellName) #update old halo with new so next will have that in it

[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [ ]:
df_halo_updated_by_cut_1

In [ ]:
 df_8_Life

,Date,Time From Cut-in,Time from Bath-Up,CT_DATE,CELL_START_DATE,CT_CELL_ID,CT_BEAM_POS,CT_FEED_RATE,CT_SM_RES,CT_BRSP,...,Total Given Energy,Cathode Temperature (T/E),Cathode Temperature (D/E),Collector Bar Temperature,Preheat Mean Volt,Preheat Mean Current,Anode CV %,PAH Model,Technology,Factory
0,2020-05-29 06:53:00,-1,-1,2020-05-29T06:53:00.0000000,2020-06-01T06:52:00.0000000,1A008,0,100,1.17,5.82,...,59811,935,865,184.465,2.53,421.5,16.5,Eco-Friendly Semi-Graphitic,DX,JA
1,2020-05-29 06:54:00,-1,-1,2020-05-29T06:54:00.0000000,2020-06-01T06:52:00.0000000,1A008,0,100,1.17,5.82,...,59811,935,865,184.465,2.53,421.5,16.5,Eco-Friendly Semi-Graphitic,DX,JA
2,2020-05-29 06:55:00,-1,-1,2020-05-29T06:55:00.0000000,2020-06-01T06:52:00.0000000,1A008,0,100,1.17,5.82,...,59811,935,865,184.465,2.53,421.5,16.5,Eco-Friendly Semi-Graphitic,DX,JA
3,2020-05-29 06:56:00,-1,-1,2020-05-29T06:56:00.0000000,2020-06-01T06:52:00.0000000,1A008,0,100,1.17,5.82,...,59811,935,865,184.465,2.53,421.5,16.5,Eco-Friendly Semi-Graphitic,DX,JA
4,2020-05-29 06:57:00,-1,-1,2020-05-29T06:57:00.0000000,2020-06-01T06:52:00.0000000,1A008,0,100,1.17,5.82,...,59811,935,865,184.465,2.53,421.5,16.5,Eco-Friendly Semi-Graphitic,DX,JA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4949,2020-06-01 16:25:35.535000,-1,-1,2020-05-29T08:10:00.0000056,2020-06-01T06:52:00.0000000,1A008,0,100,1.148593,5.82,...,59811,935,865,184.465,2.53,421.5,16.5,Eco-Friendly Semi-Graphitic,DX,JA
4950,2020-06-01 16:26:35.530000,-1,-1,2020-05-29T08:11:00.0000056,2020-06-01T06:52:00.0000000,1A008,0,100,1.148589,5.82,...,59811,935,865,184.465,2.53,421.5,16.5,Eco-Friendly Semi-Graphitic,DX,JA
4951,2020-06-01 16:27:35.525000,-1,-1,2020-05-29T08:12:00.0000056,2020-06-01T06:52:00.0000000,1A008,0,100,1.148585,5.82,...,59811,935,865,184.465,2.53,421.5,16.5,Eco-Friendly Semi-Graphitic,DX,JA
4952,2020-06-01 16:28:35.520000,-1,-1,2020-05-29T08:13:00.0000056,2020-06-01T06:52:00.0000000,1A008,0,100,1.148581,5.82,...,59811,935,865,184.465,2.53,421.5,16.5,Eco-Friendly Semi-Graphitic,DX,JA


In [ ]:
df_halo_updated[df_halo_updated["CT_CELL_ID"] == cellName]

,TAGS 2,TAGS,Date,Time From Cut-in,Time from Bath-Up,CT_DATE,CELL_START_DATE,CT_CELL_ID,CT_BEAM_POS,CT_FEED_RATE,...,1A008 Bath up Forecast,1A008 Temperature,1A008 PH Temerature,1A008 TGT,1A008 TMT,Voltage,Paste Type,Collector Bar,Anode CV %,Technology
261213,-1_1A008,2020-06-01 16:29:35.515000_1A008,2020-05-29 06:53:00,-1,-1,2020-05-29T06:53:00.0000000,2020-06-01T06:52:00.0000000,1A008,0,100,...,939.737938,657.545455,NaN,,,0.496,NaN,NaN,NaN,NaN
261214,-1_1A008,2020-06-01 16:29:35.515000_1A008,2020-05-29 06:54:00,-1,-1,2020-05-29T06:54:00.0000000,2020-06-01T06:52:00.0000000,1A008,0,100,...,939.737938,657.545455,NaN,,,0.496,NaN,NaN,NaN,NaN
261215,-1_1A008,2020-06-01 16:29:35.515000_1A008,2020-05-29 06:55:00,-1,-1,2020-05-29T06:55:00.0000000,2020-06-01T06:52:00.0000000,1A008,0,100,...,939.737938,657.545455,NaN,,,0.496,NaN,NaN,NaN,NaN
261216,-1_1A008,2020-06-01 16:29:35.515000_1A008,2020-05-29 06:56:00,-1,-1,2020-05-29T06:56:00.0000000,2020-06-01T06:52:00.0000000,1A008,0,100,...,939.737938,657.545455,NaN,,,0.496,NaN,NaN,NaN,NaN
261217,-1_1A008,2020-06-01 16:29:35.515000_1A008,2020-05-29 06:57:00,-1,-1,2020-05-29T06:57:00.0000000,2020-06-01T06:52:00.0000000,1A008,0,100,...,939.737938,657.545455,NaN,,,0.496,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276070,NaN,NaN,2020-06-01 16:25:35.535000,-1,-1,2020-05-29T08:10:00.0000056,2020-06-01T06:52:00.0000000,1A008,0,100,...,NaN,NaN,NaN,NaN,NaN,NaN,GC,SL_REC_SQ_CU,16.5,DX
276071,NaN,NaN,2020-06-01 16:26:35.530000,-1,-1,2020-05-29T08:11:00.0000056,2020-06-01T06:52:00.0000000,1A008,0,100,...,NaN,NaN,NaN,NaN,NaN,NaN,GC,SL_REC_SQ_CU,16.5,DX
276072,NaN,NaN,2020-06-01 16:27:35.525000,-1,-1,2020-05-29T08:12:00.0000056,2020-06-01T06:52:00.0000000,1A008,0,100,...,NaN,NaN,NaN,NaN,NaN,NaN,GC,SL_REC_SQ_CU,16.5,DX
276073,NaN,NaN,2020-06-01 16:28:35.520000,-1,-1,2020-05-29T08:13:00.0000056,2020-06-01T06:52:00.0000000,1A008,0,100,...,NaN,NaN,NaN,NaN,NaN,NaN,GC,SL_REC_SQ_CU,16.5,DX


In [ ]:
d20 = pd.read_excel("/content/drive/MyDrive/nitesh /20.xlsx",sheet_name = "Action List" );

In [ ]:
d20.head()

,TAGS,Date,Time From Cut-in,Time From Bath-up,CT_DATE,CELL_START_DATE,CT_CELL_ID,CD_OFFLINE,CD_POT_AGE,CD_REST_POT_AGE,...,CD_XS_ALF3,CD_SPAR,CD_CRYOLITE_ADDED,CD_LIQUID_BATH_ADDED,CD_METAL_ADDED,CD_LIQUID_METAL_ADDED,CD_POTLINE_TAP_WT,CD_NO_AE,Total Liquid Before Tap,Total Liquid After Tap
0,43849.9104166667_1A020,2020-01-19 21:51:00,-1052,NaN,2020-01-19T21:51:00.0000000,2020-01-22T21:50:00.0000000,1A020,2020-01-19T23:00:00.0000000,1545.0,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,26000.0,NaN,0,0
1,43849.9111111111_1A020,2020-01-19 21:52:00,-1051,NaN,2020-01-19T21:52:00.0000000,2020-01-22T21:50:00.0000000,1A020,2020-01-19T23:00:00.0000000,1545.0,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,26000.0,NaN,0,0
2,43849.9118055556_1A020,2020-01-19 21:53:00,-1050,NaN,2020-01-19T21:53:00.0000000,2020-01-22T21:50:00.0000000,1A020,2020-01-19T23:00:00.0000000,1545.0,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,26000.0,NaN,0,0
3,43849.9125_1A020,2020-01-19 21:54:00,-1049,NaN,2020-01-19T21:54:00.0000000,2020-01-22T21:50:00.0000000,1A020,2020-01-19T23:00:00.0000000,1545.0,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,26000.0,NaN,0,0
4,43849.9131944445_1A020,2020-01-19 21:55:00,-1048,NaN,2020-01-19T21:55:00.0000000,2020-01-22T21:50:00.0000000,1A020,2020-01-19T23:00:00.0000000,1545.0,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,26000.0,NaN,0,0


In [ ]:
df_8_ActionList

,Date,Time From Cut-in,Time From Bath-up,CT_DATE,CELL_START_DATE,CT_CELL_ID,CD_OFFLINE,CD_POT_AGE,CD_REST_POT_AGE,CD_BATH_TEMP,...,CD_XS_ALF3,CD_SPAR,CD_CRYOLITE_ADDED,CD_LIQUID_BATH_ADDED,CD_METAL_ADDED,CD_LIQUID_METAL_ADDED,CD_POTLINE_TAP_WT,CD_NO_AE,Total Liquid Before Tap,Total Liquid After Tap
0,2020-05-29 06:53:00,-931,NaN,2020-05-29T06:53:00.0000000,2020-06-01T06:52:00.0000000,8,2020-05-29T23:00:00.0000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,2020-05-29 06:54:00,-930,NaN,2020-05-29T06:54:00.0000000,2020-06-01T06:52:00.0000000,8,2020-05-29T23:00:00.0000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2020-05-29 06:55:00,-929,NaN,2020-05-29T06:55:00.0000000,2020-06-01T06:52:00.0000000,8,2020-05-29T23:00:00.0000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,2020-05-29 06:56:00,-928,NaN,2020-05-29T06:56:00.0000000,2020-06-01T06:52:00.0000000,8,2020-05-29T23:00:00.0000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,2020-05-29 06:57:00,-927,NaN,2020-05-29T06:57:00.0000000,2020-06-01T06:52:00.0000000,8,2020-05-29T23:00:00.0000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5049,2020-06-01 19:08:00,4124,NaN,2020-06-01T19:08:00.0000000,2020-06-01T06:52:00.0000000,8,2020-06-01T23:00:00.0000000,1.0,NaN,1001.0,...,NaN,NaN,NaN,12000.0,NaN,12180.0,NaN,NaN,0,0
5050,2020-06-01 19:09:00,4125,NaN,2020-06-01T19:09:00.0000000,2020-06-01T06:52:00.0000000,8,2020-06-01T23:00:00.0000000,1.0,NaN,1001.0,...,NaN,NaN,NaN,12000.0,NaN,12180.0,NaN,NaN,0,0
5051,2020-06-01 19:10:00,4126,NaN,2020-06-01T19:10:00.0000000,2020-06-01T06:52:00.0000000,8,2020-06-01T23:00:00.0000000,1.0,NaN,1001.0,...,NaN,NaN,NaN,12000.0,NaN,12180.0,NaN,NaN,0,0
5052,2020-06-01 19:11:00,4127,NaN,2020-06-01T19:11:00.0000000,2020-06-01T06:52:00.0000000,8,2020-06-01T23:00:00.0000000,1.0,NaN,1001.0,...,NaN,NaN,NaN,12000.0,NaN,12180.0,NaN,NaN,0,0


In [ ]:
index = list(df_8_ActionList.columns).index("CD_POT_AGE")
df_8_ActionList.iloc[:,index:index + 18]

,CD_POT_AGE,CD_REST_POT_AGE,CD_BATH_TEMP,CD_BATH_DEPTH_BF_TAP,CD_BATH_DEPTH_AF_TAP,CD_METAL_DEPTH_BF_TAP,CD_METAL_DEPTH_AF_TAP,CD_ALF3_ADDED,CD_NA2CO3_ADDED,CD_CAF2_ADDED,CD_XS_ALF3,CD_SPAR,CD_CRYOLITE_ADDED,CD_LIQUID_BATH_ADDED,CD_METAL_ADDED,CD_LIQUID_METAL_ADDED,CD_POTLINE_TAP_WT,CD_NO_AE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5049,1.0,NaN,1001.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,12000.0,NaN,12180.0,NaN,NaN
5050,1.0,NaN,1001.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,12000.0,NaN,12180.0,NaN,NaN
5051,1.0,NaN,1001.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,12000.0,NaN,12180.0,NaN,NaN
5052,1.0,NaN,1001.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,12000.0,NaN,12180.0,NaN,NaN


In [ ]:
df_halo_updated_PGS

,TAGS,Time From Cut-In,Time From Bath Up,Cell,Date - Time,Time,PH_ENERGY,Calc Energy,TEMP_TGT,27-C\nBK 1,21-C\nBK 2,14-C\nBK 3,8-C\nBK 4,2-C\nBK 5,Time From Cut-in,Time Before the Bath-up,Average,Watt/HR to Celsius,Watt/HR to Energy/HR,Forecast PH TMP,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,513_1A036,513,-2870.0,1A036,2020-05-31 11:10:00,9.0,11204.998547,14.894331,NaN,244.0,262.0,249.0,276.0,282.0,513.0,-2870.0,263.0,498.94,945360.0,263.000000,NaN,NaN,NaN,NaN,6085.766414
1,822_1A036,822,-2560.0,1A036,2020-05-31 16:20:00,14.0,17290.764961,20.960828,NaN,358.0,377.0,370.0,383.0,390.0,823.0,-2560.0,374.0,713.64,1352160.0,374.000000,NaN,NaN,NaN,NaN,4935.535499
2,1082_1A036,1082,-2300.0,1A036,2020-05-31 20:40:00,18.0,22226.300460,25.896919,NaN,470.0,485.0,468.0,495.0,440.0,1083.0,-2300.0,455.0,896.04,1697760.0,455.000000,NaN,NaN,NaN,NaN,4232.485000
3,1313_1A036,1313,-2070.0,1A036,2020-06-01 00:30:00,22.0,26458.785461,30.148118,NaN,545.0,548.0,545.0,570.0,526.0,1313.0,-2070.0,535.5,1038.92,1968480.0,535.500000,NaN,NaN,NaN,NaN,4306.327020
4,1553_1A036,1553,-1830.0,1A036,2020-06-01 04:30:00,26.0,30765.112480,34.454445,NaN,620.0,626.0,620.0,650.0,590.0,1553.0,-1830.0,605.0,1180.28,2236320.0,605.000000,NaN,NaN,NaN,NaN,3804.533302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,3462_1A120,3462,-1031.0,1A120,2020-07-12 08:50:00,35.0,40874.751948,NaN,NaN,750.0,835.0,742.0,796.0,714.0,2091.0,NaN,767.4,1458.06,2762640.0,767.400000,NaN,NaN,NaN,NaN,NaN
815,3781_1A120,3781,-851.0,1A120,2020-07-12 11:50:00,38.0,44119.754063,NaN,NaN,777.0,872.0,802.0,840.0,755.0,2271.0,NaN,809.2,1537.48,2913120.0,809.200000,NaN,NaN,NaN,NaN,NaN
816,4101_1A120,4101,-561.0,1A120,2020-07-12 16:40:00,43.0,49392.938700,NaN,NaN,837.0,930.0,858.0,884.0,812.0,2561.0,NaN,864.2,1641.98,3111120.0,864.200000,NaN,NaN,NaN,NaN,NaN
817,4421_1A120,4421,-181.0,1A120,2020-07-12 23:00:00,49.0,56441.834011,NaN,NaN,872.0,960.0,900.0,920.0,848.0,2941.0,NaN,900.0,1710.00,3240000.0,900.000000,NaN,NaN,NaN,NaN,NaN
